In [2]:
%pip install tensorflow-datasets

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 1.1 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 2.6 MB/s eta 0:00:0000:0100:01
Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Created wheel for dm-tree: filename=dm_tree-0.1.9-cp312-cp312-macosx_15_0_universal2.whl size=112039 sha256=552751569475c2ea05c11882c905db767ecfbc39358edba4affe0b965f2a1619
  Stored in directory: /Users/nehasharma/Library/Caches/pip/wheels/89/01/af/d33b8877c63c2b90659a0eba4cffefbd049665c311d0a3d23a
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21548 sha256=a10b6dffa87d738aca938102ff540fad9a5d1c5614683797a0c3e5ae97837501
  Stored in directory: /Users/nehasharma/Library/Caches/pip/wheels/e7/e6/28/864bdfee5339dbd6ddcb5a186286a8e217648ec198bdf0097d
Successfully built dm-tree promise

[notice]

Objective:
To implement a three-layer neural network using TensorFlow (without Keras) for classifying handwritten digits from the MNIST dataset. The implementation should demonstrate feed-forward and back-propagation approaches.

Description of the Model:
The model consists of three layers:
Input Layer: 784 neurons (28x28 flattened image pixels)
Hidden Layer 1: 128 neurons, activation: sigmoid
Hidden Layer 2: 64 neurons, activation: sigmoid
Output Layer: 10 neurons (digits 0-9), producing logits
Feed-forward propagation computes the output.
Back-propagation updates weights using Stochastic Gradient Descent (SGD) with Momentum.
The loss function used is Softmax Cross-Entropy.

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
dataset, info = tfds.load("mnist", as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset["train"], dataset["test"]

2025-02-22 16:36:46.692786: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 100%|██████████| 5/5 [00:14<00:00,  2.81s/ file]

Dataset mnist downloaded and prepared to /Users/nehasharma/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [5]:
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0 
    image = tf.reshape(image, [-1])  
    label = tf.one_hot(label, depth=10)  
    return image, label

train_dataset = train_dataset.map(preprocess).shuffle(10000).batch(BATCH_SIZE)
test_dataset = test_dataset.map(preprocess).batch(BATCH_SIZE)


In [6]:

# Define model parameters
input_dim = 784
hidden_dim1 = 128
hidden_dim2 = 64
output_dim = 10

# Initialize weights and biases
W1 = tf.Variable(tf.random.normal([input_dim, hidden_dim1], stddev=0.1))
b1 = tf.Variable(tf.zeros([hidden_dim1]))
W2 = tf.Variable(tf.random.normal([hidden_dim1, hidden_dim2], stddev=0.1))
b2 = tf.Variable(tf.zeros([hidden_dim2]))
W3 = tf.Variable(tf.random.normal([hidden_dim2, output_dim], stddev=0.1))
b3 = tf.Variable(tf.zeros([output_dim]))

In [7]:
# Define forward pass
def model(x):
    hidden_layer1 = tf.sigmoid(tf.matmul(x, W1) + b1)  # First Hidden Layer
    hidden_layer2 = tf.sigmoid(tf.matmul(hidden_layer1, W2) + b2)  # Second Hidden Layer
    logits = tf.matmul(hidden_layer2, W3) + b3  # Output layer (logits)
    return logits

In [8]:
# Loss function (Softmax Cross-Entropy)
def compute_loss(logits, labels):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))

# Accuracy function
def compute_accuracy(dataset):
    correct_preds, total_samples = 0, 0
    for images, labels in dataset:
        logits = model(images)
        correct_preds += tf.reduce_sum(tf.cast(tf.equal(tf.argmax(logits, axis=1), tf.argmax(labels, axis=1)), tf.float32)).numpy()
        total_samples += images.shape[0]
    return correct_preds / total_samples

# Optimizer (SGD with Momentum)
optimizer = tf.optimizers.SGD(learning_rate=0.1, momentum=0.9)

In [9]:

# Training step function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        logits = model(images)
        loss = compute_loss(logits, labels)
    gradients = tape.gradient(loss, [W1, b1, W2, b2, W3, b3])
    optimizer.apply_gradients(zip(gradients, [W1, b1, W2, b2, W3, b3]))
    return loss

In [10]:
# Training loop
epochs = 10
for epoch in range(epochs):
    total_loss = 0.0
    for images, labels in train_dataset:
        loss = train_step(images, labels)
        total_loss += loss.numpy()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

2025-02-22 16:39:08.403789: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-02-22 16:39:18.180031: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1, Loss: 671.4664


2025-02-22 16:39:27.719698: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2, Loss: 254.0650
Epoch 3, Loss: 172.3769


2025-02-22 16:39:46.985768: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 4, Loss: 128.9476
Epoch 5, Loss: 103.9082
Epoch 6, Loss: 81.9009
Epoch 7, Loss: 64.9891


2025-02-22 16:40:24.741564: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 8, Loss: 52.1213
Epoch 9, Loss: 43.1821
Epoch 10, Loss: 34.6094


In [11]:
# Compute final training and test accuracy
train_accuracy = compute_accuracy(train_dataset)
test_accuracy = compute_accuracy(test_dataset)

print(f"Final Training Accuracy (SGD): {train_accuracy:.4f}")
print(f"Final Test Accuracy (SGD): {test_accuracy:.4f}")

Final Training Accuracy (SGD): 0.9963
Final Test Accuracy (SGD): 0.9781


Description of the Code:

Dataset Preprocessing:
Loads MNIST dataset using tensorflow_datasets.
Normalizes images (0 to 1 range).
Flattens 28×28 images into 784 feature vectors.
Converts labels into one-hot encoding (for softmax classification).
Uses shuffling and batching for efficient training.

Model Architecture:
Weight matrices (W1, W2, W3) initialized randomly.
Bias vectors (b1, b2, b3) initialized to zeros.
Activation function: Sigmoid for hidden layers, Softmax for output.

Feed-forward computation:
First hidden layer: sigmoid(W1*x + b1)
Second hidden layer: sigmoid(W2*hidden1 + b2)
Output layer (logits): W3*hidden2 + b3

Training Process:
Loss function: softmax_cross_entropy_with_logits
Optimizer: Stochastic Gradient Descent (SGD) with momentum (0.9)
Gradient computation using tf.GradientTape().
Backpropagation updates weights (W1, W2, W3) and biases.

Performance Evaluation:
Computes accuracy on training and test data.
Outputs loss per epoch.

Conclusion:
Successfully implemented a three-layer neural network using TensorFlow (without Keras).
Demonstrated feed-forward propagation and backpropagation for training.
Achieved reasonable accuracy on the MNIST dataset using SGD with Momentum.
The model can be further improved using ReLU activations, batch normalization, or Adam optimizer.